In [1]:
import numpy as np
import pandas as pd
import json

import gurobipy as gb
from gurobipy import GRB

In [2]:
c = pd.read_json('./cleandata/dist_mat.json')
t = pd.read_json('./cleandata/time_mat.json')
locdem = pd.read_excel('./cleandata/locdem.xlsx')
#8 - 124,11 - 125
c = c.loc[:123, :123].to_numpy()
t = t.loc[:123, :123].to_numpy()
q = locdem['Number of pallets']
q = q.to_numpy()

def get(i):
    if i == 124:
        return 8
    if i == 125:
        return 11
    return i

def time(i,j):
    i,j = get(i), get(j)
    return t[i,j]

def cost(i,j):
    if (i,j) in [(125,11), (11,125), (8,124), (124,8)]:
        return 1e+8
    i,j = get(i), get(j)
    return c[i,j]

In [3]:
v = 126
V = set(range(126))
def Del(p):
    return V - set([p])
N = Del(0)
Q = 9

from itertools import product
A = filter(lambda tup: tup[0] != tup[1], product(N,N))

In [4]:
m1 = gb.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12


In [5]:
#Remember to take care of 124,125
#8 - 124,11 - 125

In [6]:
x = m1.addVars(v,v,vtype=GRB.BINARY, name='x')
m1.setObjective(sum([cost(i,j) * x[i,j] for i,j in product(V,V)]), GRB.MINIMIZE)

In [7]:
#Source and sink constraints
for i in N:
    m1.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
for j in N:
    m1.addConstr(sum([x[i,j] for i in Del(j)]) == 1 )

In [8]:
#Construction of L loops
L = m1.addVar(lb=0.0, vtype=GRB.INTEGER, name='L')
m1.addConstr(sum([x[0,j] for j in Del(0)]) == L)

<gurobi.Constr *Awaiting Model Update*>

In [9]:
#Capacity constraints
u = m1.addVars(V, vtype=GRB.INTEGER, name='u')
for i,j in A:
    m1.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
for i in V:
    m1.addConstr(u[i] >= q[i])

In [10]:
m1.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 15877 rows, 16003 columns and 78002 nonzeros
Model fingerprint: 0xff615317
Variable types: 0 continuous, 16003 integer (15876 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [2e+01, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 9e+00]
Found heuristic solution: objective 1.180264e+07
Presolve removed 127 rows and 128 columns
Presolve time: 0.26s
Presolved: 15750 rows, 15875 columns, 77750 nonzeros
Variable types: 0 continuous, 15875 integer (15750 binary)

Root relaxation: objective 1.057921e+06, 2489 iterations, 1.43 seconds (0.98 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1057921.39    0  303 1.1803e+07 1057921.39  91.0%     -    2s
H 

In [13]:
STR = m1.getJSONSolution()

In [20]:
json.dump(json.loads(STR), open('./globalsoln.json','w'), indent=4)

In [22]:
STR = json.loads(STR)

In [23]:
STR['Vars']

[{'VarName': 'x[0,9]', 'X': 1},
 {'VarName': 'x[0,10]', 'X': 1},
 {'VarName': 'x[0,13]', 'X': 1},
 {'VarName': 'x[0,14]', 'X': 1},
 {'VarName': 'x[0,15]', 'X': 1},
 {'VarName': 'x[0,17]', 'X': 1},
 {'VarName': 'x[0,18]', 'X': 1},
 {'VarName': 'x[0,25]', 'X': 1},
 {'VarName': 'x[0,26]', 'X': 1},
 {'VarName': 'x[0,32]', 'X': 1},
 {'VarName': 'x[0,47]', 'X': 1},
 {'VarName': 'x[0,48]', 'X': 1},
 {'VarName': 'x[0,49]', 'X': 1},
 {'VarName': 'x[0,53]', 'X': 1},
 {'VarName': 'x[0,55]', 'X': 1},
 {'VarName': 'x[0,59]', 'X': 1},
 {'VarName': 'x[0,66]', 'X': 1},
 {'VarName': 'x[0,85]', 'X': 1},
 {'VarName': 'x[0,96]', 'X': 1},
 {'VarName': 'x[0,97]', 'X': 1},
 {'VarName': 'x[0,98]', 'X': 1},
 {'VarName': 'x[0,99]', 'X': 1},
 {'VarName': 'x[0,100]', 'X': 1},
 {'VarName': 'x[0,101]', 'X': 1},
 {'VarName': 'x[0,102]', 'X': 1},
 {'VarName': 'x[0,103]', 'X': 1},
 {'VarName': 'x[0,104]', 'X': 1},
 {'VarName': 'x[0,105]', 'X': 1},
 {'VarName': 'x[0,106]', 'X': 1},
 {'VarName': 'x[0,107]', 'X': 1},
 {'

In [26]:
m1.


Solution quality statistics for model Unnamed :
  Maximum violation:
    Bound       : 0.00000000e+00
    Constraint  : 0.00000000e+00
    Integrality : 0.00000000e+00
